In [138]:
import numpy as np
import itertools
import re
import copy

In [139]:
registers = {}
memory = []
pointer = 0

with open('input.txt') as f:
    regex = 'Register ([ABC]): ([0-9]*)'
    for line in itertools.takewhile(lambda x: x != '\n', f):
        match = re.match(regex, line)
        registers[match.group(1)] = int(match.group(2))
    regex = 'Program: (.*)'
    for line in f:
        match = re.match(regex, line)
        for byte in match.group(1).split(','):
            memory.append(int(byte))

#memory = memory[:14]

initial_memory = copy.deepcopy(memory)
initial_registers = copy.deepcopy(registers)

print(pointer)
print(memory)
print(registers)

0
[2, 4, 1, 3, 7, 5, 0, 3, 4, 1, 1, 5, 5, 5, 3, 0]
{'A': 45483412, 'B': 0, 'C': 0}


In [292]:
'''
    Combo operands 0 through 3 represent literal values 0 through 3.
    Combo operand 4 represents the value of register A.
    Combo operand 5 represents the value of register B.
    Combo operand 6 represents the value of register C.
    Combo operand 7 is reserved and will not appear in valid programs.
'''
def get_combo_operand(operand):
    if operand >= 0 and operand <= 3:
        return operand
    elif operand == 4:
        #print(f"combo A {registers['A']} ", end="")
        return registers['A']
    elif operand == 5:
        #print(f"combo B {registers['B']} ", end="")
        return registers['B']
    elif operand == 6:
        #print(f"combo C {registers['C']} ", end="")
        return registers['C']
    elif operand == 7:
        raise ValueError

In [293]:
def execute_instruction():
    global pointer
    opcode = memory[pointer]
    operand = memory[pointer+1]
    pointer += 2

    #print(f"ptr {pointer} opc {opcode} opr {operand} ", end="")
    
    if opcode == 0 or opcode == 2 or opcode == 5 or opcode == 6 or opcode == 7:
        operand = get_combo_operand(operand)

    
    
    if opcode == 0:
        '''
        The adv instruction (opcode 0) performs division.
        The numerator is the value in the A register.
        The denominator is found by raising 2 to the power of the instruction's combo operand.
        (So, an operand of 2 would divide A by 4 (2^2); an operand of 5 would divide A by 2^B.)
        The result of the division operation is truncated to an integer and then written to the A register.
        '''
        val = int(registers['A'] / (np.pow(2, operand)))
        #print(f"A=int(A/(2^op))=int({registers['A']}/({2}**{operand}))={val}")
        registers['A'] = val
    elif opcode == 1:
        '''
        The bxl instruction (opcode 1) calculates the bitwise XOR of register B and the instruction's literal operand,
        then stores the result in register B.
        '''
        val = registers['B'] ^ operand
        #print(f"B=B^op={registers['B']}^{operand}={val}")
        registers['B'] = val
    elif opcode == 2:
        '''
        The bst instruction (opcode 2) calculates the value of its combo operand modulo 8
        (thereby keeping only its lowest 3 bits), then writes that value to the B register.
        '''
        val = operand % 8
        #print(f"B=op%8={operand}%8={val}")
        registers['B'] = val
    elif opcode == 3:
        '''
        The jnz instruction (opcode 3) does nothing if the A register is 0.
        However, if the A register is not zero, it jumps by setting the instruction pointer
        to the value of its literal operand; if this instruction jumps, the instruction pointer
        is not increased by 2 after this instruction.
        '''
        if registers['A'] != 0:
            #print(f"jmp op {operand}")
            pointer = operand
        else:
            #print(f"no-jmp")
            pass
    elif opcode == 4:
        '''
        The bxc instruction (opcode 4) calculates the bitwise XOR of register B and register C,
        then stores the result in register B. (For legacy reasons, this instruction reads an operand
        but ignores it.)
        '''
        val = registers['B'] ^ registers['C']
        #print(f"B=B^C={registers['B']}^{registers['C']}={val}")
        registers['B'] = val
    elif opcode == 5:
        '''
        The out instruction (opcode 5) calculates the value of its combo operand modulo 8,
        then outputs that value. (If a program outputs multiple values, they are separated by commas.)
        '''
        val = operand % 8
        #print(f"output op%8={val}")
        outputs.append(val)
    elif opcode == 6:
        '''
        The bdv instruction (opcode 6) works exactly like the adv instruction except that the
        result is stored in the B register. (The numerator is still read from the A register.)
        '''
        val = int(registers['A'] / (np.pow(2, operand)))
        #print(f"B=int(A/(2^op))=int({registers['A']}/({2}**{operand}))={val}")
        registers['B'] = val
    elif opcode == 7:
        '''
        The cdv instruction (opcode 7) works exactly like the adv instruction except that the
        result is stored in the C register. (The numerator is still read from the A register.)
        '''
        val = int(registers['A'] / (np.pow(2, operand)))
        #print(f"C=int(A/(2^op))=int({registers['A']}/({2}**{operand}))={val}")
        registers['C'] = val

In [654]:
#lin_start = 216000000000000
#lin_end   = 240000000000000

#lin_start = 218383249000000
#lin_end   = 218383400000000

#lin_start = 235975000000000
#lin_end   = 236590000000000

#lin_start = 236581108600000
#lin_end   = 236581108700000


# 2,4,1,3,7,5,0,3,4,1,1,5,5,5,3,0
compare = [2,4,1,3,7,5,0,3,4,1,1,5,5,5,3,0]
start_a = 216184226054148
a_s = []
#step = 8**3-1
step = 120000-8-1-13-144-3
step *= 100
num_steps = 20000
start_a -= (num_steps//2)*step
#for a in np.arange(start_a, start_a + num_steps*step, step):
lin_start = 236581108660000
lin_end   = 236581108680000
            
#lin_start = 218383249000000
#lin_end   = 218383400000000

print(lin_start, lin_end)
for a in np.linspace(lin_start, lin_end, 10000):
    a = int(a)
    if ((((a%8)^3)^(int(a/(2**((a%8)^3)))))^5)%8!=2:
        continue
    outputs = []
    pointer = 0
    #registers['A'] = initial_registers['A']
    registers['A'] = int(a)
    registers['B'] = initial_registers['B']
    registers['C'] = initial_registers['C']
    while pointer < len(memory):
        execute_instruction()
    if len(outputs) != 16:
        continue
    if outputs[16-len(compare):]==compare:
        print(a, outputs)
        a_s.append(int(a))
a_s = np.array(a_s)
print(a_s[:30])
#print([np.base_repr(a, base=8) for a in a_s])
#print((a_s - a_s[0])/step)

236581108660000 236581108680000
236581108670061 [2, 4, 1, 3, 7, 5, 0, 3, 4, 1, 1, 5, 5, 5, 3, 0]
236581108670143 [2, 4, 1, 3, 7, 5, 0, 3, 4, 1, 1, 5, 5, 5, 3, 0]
[236581108670061 236581108670143]


In [ ]:
'''
start of correct output len
35184372088832

start of ....3,0
215504279044100


step = 35184372088
       68719476736
step = 8**5

'''